# https://ragrawal.wordpress.com/2015/10/02/spark-custom-udf-example/

## Spark: Custom UDF Example.

UDF (User Defined Functions) and UDAF (User Defined Aggregate Functions) are key components of big data languages such as Pig and Hive.  They allow to extend the language constructs to do adhoc processing on distributed datasets.  In this walkthrough, we will focus on writing a custom UDF in Spark.

As a motivating example, assume that we are given some student data containing the student's name, subject, and score.  We want to convert the numerical score into ordinal categories based on the following logic:

- A:  score >= 80.
- B:  60 <= score < 80.
- C:  35 <= score < 60.
- D:  score < 35.

Below is the relevant Python code if you are using PySpark.

#### Step 1:  Basic Python Stuff.  We are generating a random dataset.

In [1]:
# Generate Random Data
import itertools
import random

In [2]:
students = ["John", "Mike", "Matt"]
subjects = ["Math", "Science", "Geography", "History"]
random.seed(1)
data = []

for student, subject in itertools.product(students, subjects):
    data.append((student, subject, random.randint(0, 100)))

for record in data:
    print(record)

('John', 'Math', 17)
('John', 'Science', 72)
('John', 'Geography', 97)
('John', 'History', 8)
('Mike', 'Math', 32)
('Mike', 'Science', 15)
('Mike', 'Geography', 63)
('Mike', 'History', 97)
('Matt', 'Math', 57)
('Matt', 'Science', 60)
('Matt', 'Geography', 83)
('Matt', 'History', 48)


#### Step 2:  Constructing our DataFrame.

In [3]:
# Create a Schema Object.
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("student", StringType(), nullable=False),
    StructField("subject", StringType(), nullable=False),
    StructField("score", IntegerType(), nullable=False)
])

In [4]:
# Create the DataFrame
import pandas as pd
from pyspark.sql import SparkSession

In [5]:
pd_df = pd.DataFrame(data, columns=["student", "subject", "score"])
print(pd_df)

   student    subject  score
0     John       Math     17
1     John    Science     72
2     John  Geography     97
3     John    History      8
4     Mike       Math     32
5     Mike    Science     15
6     Mike  Geography     63
7     Mike    History     97
8     Matt       Math     57
9     Matt    Science     60
10    Matt  Geography     83
11    Matt    History     48


In [6]:
# Create a Spark Session.
spark = SparkSession.builder.appName("example_c").getOrCreate()

In [7]:
sdf = spark.createDataFrame(data=pd_df, schema=schema)
sdf.show()

+-------+---------+-----+
|student|  subject|score|
+-------+---------+-----+
|   John|     Math|   17|
|   John|  Science|   72|
|   John|Geography|   97|
|   John|  History|    8|
|   Mike|     Math|   32|
|   Mike|  Science|   15|
|   Mike|Geography|   63|
|   Mike|  History|   97|
|   Matt|     Math|   57|
|   Matt|  Science|   60|
|   Matt|Geography|   83|
|   Matt|  History|   48|
+-------+---------+-----+



#### Step 3:  The main part of the code dealing with defining the UDF.

In [8]:
from pyspark.sql.functions import udf

In [9]:
# Define the UDF.
def scoreToCategory(score):
    if score >= 80:
        return "A"
    elif 60 <= score < 80:
        return "B"
    elif 35 <= score < 60:
        return "C"
    else:  # i.e. score < 35
        return "D"

In [10]:
udfScoreToCategory = udf(f=scoreToCategory, returnType=StringType())

In [12]:
sdf.withColumn("category", udfScoreToCategory("score")).show()

+-------+---------+-----+--------+
|student|  subject|score|category|
+-------+---------+-----+--------+
|   John|     Math|   17|       D|
|   John|  Science|   72|       B|
|   John|Geography|   97|       A|
|   John|  History|    8|       D|
|   Mike|     Math|   32|       D|
|   Mike|  Science|   15|       D|
|   Mike|Geography|   63|       B|
|   Mike|  History|   97|       A|
|   Matt|     Math|   57|       C|
|   Matt|  Science|   60|       B|
|   Matt|Geography|   83|       A|
|   Matt|  History|   48|       C|
+-------+---------+-----+--------+



On the flip side of the coin, it is important to realise that you should only make use of UDF when absolutely necessary.

Click on the following link which redirects you to a ~5-minute youtube clip.

[Avoid UDF](https://www.youtube.com/watch?v=SHiZzB4n46A)